In [2]:
# 📌 Create the TestDB if it doesn't exist

import pyodbc

def create_test_database():
    try:
        connection_string = (
            "DRIVER={SQL Server};"
            "SERVER=172.27.6.207;"
            "DATABASE=master;"
            "UID=sa;"
            "PWD=engRPAsql2022$$"
        )
        conn = pyodbc.connect(connection_string, timeout=5, autocommit=True)
        print("✅ Connected to SQL Server (master DB)")

        cursor = conn.cursor()
        cursor.execute("IF DB_ID('TestDB') IS NULL CREATE DATABASE TestDB;")
        print("✅ Database 'TestDB' created or already exists.")
        conn.close()
    except pyodbc.Error as e:
        print("❌ Failed to create database.")
        print("Error:", e)

create_test_database()


✅ Connected to SQL Server (master DB)
✅ Database 'TestDB' created or already exists.


In [3]:
# ✅ Connect to TestDB

def get_connection():
    try:
        connection_string = (
            "DRIVER={SQL Server};"
            "SERVER=172.27.6.207;"
            "DATABASE=TestDB;"
            "UID=sa;"
            "PWD=engRPAsql2022$$"
        )
        conn = pyodbc.connect(connection_string)
        print("✅ Connected to TestDB")
        return conn
    except Exception as e:
        print("❌ Connection failed:", e)
        return None



In [4]:
# 📋 Show all tables in TestDB

def show_tables(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT TABLE_SCHEMA, TABLE_NAME 
            FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_TYPE = 'BASE TABLE'
        """)
        tables = cursor.fetchall()
        print("📋 Tables in TestDB:")
        for schema, table in tables:
            print(f"- {schema}.{table}")
    except Exception as e:
        print(f"❌ Error fetching tables: {e}")



In [5]:
# ⚙️ Generic SQL query executor

def execute_query(conn, query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        print(cursor)
        
        if cursor.description:  # SELECT query
            columns = [col[0] for col in cursor.description]
            rows = cursor.fetchall()
            return [dict(zip(columns, row)) for row in rows]
        else:
            conn.commit()
            print("✅ Query executed successfully")
            return None
    except Exception as e:
        print("❌ Error executing query:", e)
        return None


In [6]:
# Function to execute SELECT queries and print as table
import tabulate


def run_select_query(conn, query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        rows = cursor.fetchall()
        print(rows)
        
        if not rows:
            print("⚠️ No results returned.")
        else:
            print(tabulate.tabulate(rows, headers=columns, tablefmt="fancy_grid"))
    except Exception as e:
        raise e
        print("❌ Error executing SELECT query:", e)


In [9]:
# ✅ Example: Create table & insert values

conn = get_connection()



# execute_query(conn,"""
# Delete from users;

# """)

execute_query(conn,""" 
    Delete from users;
    INSERT INTO users (username, password) VALUES ('mohan', '1234');
    INSERT INTO users (username, password) VALUES ('test', 'test');

""")

run_select_query(conn, """
   select * from users;
                 
""")


show_tables(conn)


✅ Connected to TestDB
✅ Query executed successfully
[(21, '1234', 'mohan'), (22, 'test', 'test')]
╒══════╤════════════╤════════════╕
│   id │ password   │ username   │
╞══════╪════════════╪════════════╡
│   21 │ 1234       │ mohan      │
├──────┼────────────┼────────────┤
│   22 │ test       │ test       │
╘══════╧════════════╧════════════╛
📋 Tables in TestDB:
- dbo.student
- dbo.app_users
- dbo.users
